# Exploratory Data Analysis - Cleaning and Joining

1. **[Work with missing data in a Python notebook](#missing)**
2. **[Exemplar](#unicorn)**

<a id='missing'></a>
# Work with missing data in a Python notebook

In this notebook, we will be discovering and working with missing data on a dataset. 

## Objective

We will be examining lightning strike data collected by the National Oceanic and Atmospheric Association (NOAA) for the month of August 2018. There are two datasets. The first includes five columns:  

|date|center_point_geom|longitude|latitude|number_of_strikes|
|---|---|---|---|---|

The second dataset contains seven columns:

|date|zip_code|city|state|state_code|center_point_geom|number_of_strikes|
|---|---|---|---|---|---|---|  

The first dataset has two unique colums: `longitude` and `latitude`.  
The second dataset has four unique columns: `zip_code`, `city`, `state`, and `state_code`.  
There are three columns that are common between them: `date`, `center_point_geom`, and `number_of_strikes`.

We want to combine the two datasets into a single dataframe that has all of the information from both datasets. Ideally, both datasets will have the same number of entries for the same locations on the same dates. If they don't, we'll investigate which data is missing.

In [2]:
# Import statements
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
from matplotlib import pyplot as plt

In [4]:
# Read in first dataset
df = pd.read_csv('datasets/eda_missing_data_dataset1.csv')

# Print the first 5 rows of dataset 1
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'eda_missing_data_dataset1.csv'